# A Place Like Home
## IBM Coursera Capstone Project

### Installing and importing the required packages

In [1]:
!pip install folium

In [2]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1277 sha256=ceb4a56b30d6822cc7b2f9b29387d0304cc5748b45fa177847a56cfbef50cccc
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [4]:
!pip install geopy

In [14]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=eebf483c6f872523691b2704f4f5c02370c98c5f6c5eb859480197032e70c848
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [2]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize
import wget

### Obtaining Toronto data

The Toronto data is obtained from a WIkipedia page.

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, 'html5lib')

The data from the table is pulled out and put into a dataframe.

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
# @hidden_cell
CLIENT_ID = 'HNTPHHZEZKQSAHOJCKG3ENJ00KQ20MNWC2O5EI0YTRQMEZRI' 
CLIENT_SECRET = 'Z00AEGSTBTMQ300SBDDNJVQ4SCPQLDJNEG4PPHJIYV2PHY3R' 
VERSION = '20200429'
LIMIT = 100000

The latitude and logitude data for the neighborhoods are otained from a CSV file. It is joined with the above dataframe using Postalcode as a matching key. A city column is added with a singular value "Toronto" for conveniency purposes.

In [5]:
filename="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df_loc=pd.read_csv(filename, names=["PostalCode","Latitude","Longitude"],skiprows=1)
df_tor=pd.merge(df,df_loc,on='PostalCode')
df_toro=df_tor[['Borough','Neighborhood','Latitude','Longitude']]
df_toro.insert(0,'City','Toronto')
df_toro.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.753259,-79.329656
1,Toronto,North York,Victoria Village,43.725882,-79.315572
2,Toronto,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,Toronto,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Toronto,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


Obtaining Toronto geograpical coordinates.

In [7]:
address = 'Toronto, CN'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6425637, -79.38708718320467.


A map of Toronto neighborhoods is visaulized using folium.

In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_toro['Latitude'], df_toro['Longitude'], df_toro['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Obtaining New York data

The New York neighborhood data is obatined from a JSON file and inserted into a dataframe.  A city column is added with a singular value "New York" for conveniency purposes.

In [9]:
!curl -O https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
 13  113k   13 15526    0     0   5175      0  0:00:22  0:00:03  0:00:19  4023
100  113k  100  113k    0     0  28943      0  0:00:04  0:00:04 --:--:-- 26181


In [10]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [22]:
neighborhoods_data = newyork_data['features']
column_names = ['City','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
df_ny = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    df_ny = df_ny.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
df_ny['City']='New York'
df_ny.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


The shape of Toronto and New York dataframes respectively.

In [13]:
print(df_toro.shape,df_ny.shape)

(103, 5) (306, 5)


The first row data from Toronto dataframe is added to a new dataframe with the New York data.

In [30]:
df_new=df_toro.loc[0,:].to_frame().T
df_new=df_new.append(df_ny,ignore_index=True)
df_new.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.7533,-79.3297
1,New York,Bronx,Wakefield,40.8947,-73.8472
2,New York,Bronx,Co-op City,40.8743,-73.8299
3,New York,Bronx,Eastchester,40.8876,-73.8278
4,New York,Bronx,Fieldston,40.8954,-73.9056


### Top venues in all neighborhoods

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=300):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Obtaining the venues using the above function and creating a new dataframe.

In [32]:
venues = getNearbyVenues(names=df_new['Neighborhood'],latitudes=df_new['Latitude'],longitudes=df_new['Longitude'])

Parkwoods
Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough 

In [33]:
venues.shape

(4937, 7)

Number of venues returned for each neighborhood

In [34]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,17,17,17,17,17,17
Annadale,2,2,2,2,2,2
Arden Heights,5,5,5,5,5,5
Arlington,1,1,1,1,1,1
Arrochar,10,10,10,10,10,10
...,...,...,...,...,...,...
Woodhaven,12,12,12,12,12,12
Woodlawn,14,14,14,14,14,14
Woodrow,19,19,19,19,19,19


Number of unique venue categories that can be derived from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 379 uniques categories.


### Analyzing Each Neighborhood

In [37]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood'] = venues['Neighborhood'] 
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
onehot.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waste Facility,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Animal Shelter,...,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waste Facility,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,Woodhaven,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
285,Woodlawn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
286,Woodrow,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
287,Woodside,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069767,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


### A function to return top venues from each neighborhood

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Top 10 venues in each neighborhood

In [42]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']
for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Discount Store,Breakfast Spot,Donut Shop,Spa,Fried Chicken Joint,Supermarket,Bus Station,Fast Food Restaurant,Gas Station
1,Annadale,Bakery,Train Station,Women's Store,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant
2,Arden Heights,Deli / Bodega,Pharmacy,Coffee Shop,Playground,Bus Stop,Women's Store,Farmers Market,Ethiopian Restaurant,Event Service,Event Space
3,Arlington,Grocery Store,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm
4,Arrochar,Pizza Place,Deli / Bodega,Bus Stop,Liquor Store,Italian Restaurant,Cosmetics Shop,Bagel Shop,Fast Food Restaurant,Event Service,Event Space


### Cluster the neighborhoods into 6 clusters using K-Means

In [43]:
kclusters = 6
grouped_clustering = grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)
kmeans.labels_[0:10]

array([1, 1, 5, 2, 5, 1, 1, 5, 1, 1])

In [44]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged = df_new
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged.head()

,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,North York,Parkwoods,43.7533,-79.3297,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm
1,New York,Bronx,Wakefield,40.8947,-73.8472,1.0,Pharmacy,Dessert Shop,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant
2,New York,Bronx,Co-op City,40.8743,-73.8299,1.0,Fast Food Restaurant,Bus Station,Baseball Field,Gift Shop,Salon / Barbershop,Grocery Store,Restaurant,Pizza Place,Chinese Restaurant,Jazz Club
3,New York,Bronx,Eastchester,40.8876,-73.8278,1.0,Caribbean Restaurant,Fast Food Restaurant,Food & Drink Shop,Automotive Shop,Diner,Bus Station,Platform,Bus Stop,Donut Shop,Women's Store
4,New York,Bronx,Fieldston,40.8954,-73.9056,1.0,Plaza,River,Bus Station,Women's Store,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor


Locating in which cluster the Toronto neighborhood has been placed. 

In [50]:
merged.loc[merged['City']=='Toronto']

,City,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,North York,Parkwoods,43.7533,-79.3297,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm


The cluster where the selected Toronto neighborhood is categorized along with the New York neighborhoods

In [51]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm
36,Bronx,3.0,Park,Thai Restaurant,Women's Store,Fountain,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant
136,Queens,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm
188,Queens,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm
241,Staten Island,3.0,Park,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Eye Doctor,Factory,Falafel Restaurant,Farm


### Visalizing that particular cluster

In [54]:
similar_neighborhoods=df_new.loc[[36,136,188,241],:]
similar_neighborhoods.head()

,City,Borough,Neighborhood,Latitude,Longitude
36,New York,Bronx,Spuyten Duyvil,40.8814,-73.9172
136,New York,Queens,Forest Hills,40.7253,-73.8445
188,New York,Queens,Lindenwood,40.6639,-73.8496
241,New York,Staten Island,Rossville,40.5494,-74.2157


In [56]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(lat, long))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


In [58]:
map_similar_neighborhoods = folium.Map(location=[lat, long], zoom_start=11)
for lat, lng, label in zip(similar_neighborhoods['Latitude'], similar_neighborhoods['Longitude'], similar_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_similar_neighborhoods)  
    
map_similar_neighborhoods